# How to Use Feature Extraction on Tabular Data for Machine Learning

> *Machine learning predictive modeling performance is only as good as your data, and your data is only as good as the way you prepare it for modeling.*

La técnica más común para preparar los datos consiste en estudiar un *dataset* y revisar las expectativas de un algoritmo de *machine learning*, para luego elegir cuidadosamente las técnicas de preparación de datos más adecuadas. Esto es lento, caro y requiere mucha experiencia.

Una alternativa es aplicar un conjunto de técnicas de preparación comunes en paralelo y combinar los resultados de todas las transformaciones en un solo *dataset*, para poder entrenar y evaluar un modelo.



## 1. Feature Extraction Technique for Data Preparation

Una técnica que busca el punto medio entre las descritas anteriormente, consiste en tratar la transformación de los datos de entrada como un procedimiento de **feature engineering** o **feature extraction**. Permite, entre otros beneficios, encontrar soluciones poco intuitivas a un costo computacional mucho más bajo.

### Dataset and Performance Baseline

#### Wine Classification Dataset

Se utilizará el *dataset Wine.csv*, que contiene 13 variables de entrada que describan la composición química de las muestras de vino, y requiere clasificar cada muestra en uno de tres tipos de vino posibles.

El siguiente código carga el *dataset*, lo separa en columnas de entrada y salida, y luego condensa los arreglos de datos.

In [1]:
# example of loading and summarizing the wine dataset
from pandas import read_csv
# define the location of the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'
# load the dataset as a data frame
df = read_csv(url, header=None)
# retrieve the numpy array
data = df.values
# split the columns into input and output variables
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the loaded data
print(X.shape, y.shape)

(178, 13) (178,)


Al correr el ejemplo, se puede ver que el *dataset* se cargó correctamente y que hay 179 filas de datos con 13 variables de entrada y 1 de salida. 

Seguidamente, se evalúa el modelo para determinar el *baseline performance*.

#### Baseline Model Performance

Se puede establecer un *baseline* en la tarea de clasificación de vino al evaluar un modelo en los datos *raw*. En este caso, se evaluará un modelo de regresión logística.

Primero, se utiliza la librería *scikit-learn* para garantizar que las variables de entrada son numéricas y la variable de salida es una etiqueta, como lo espera la librería. Luego, se define el modelo; se evaluará usando el estándar de *repeated stratified k-fold cross validation* con *10 folds* y *3 repeats*.

El rendimiento del modelo se evaluará utilizando la precisión en la clasificación. Al final, se reporta la media y la desviación estándar del rendimiento alcanzado.

In [2]:
# baseline model performance on the wine dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'
df = read_csv(url, header=None)
data = df.values
X, y = data[:, :-1], data[:, -1]
# minimally prepare dataset
X = X.astype('float')
y = LabelEncoder().fit_transform(y.astype('str'))
# define the model
model = LogisticRegression(solver='liblinear')
# define the cross-validation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.955 (0.049)


Se alcanzó una precisión de 95.5 porciento; esta será nuestra *baseline* en el *performance*. 

## 3. Feature Extraction Approach to Data Preparation

Puesto que las variables de entrada son numéricas, se utilizará un rango de transformaciones para cambiar su escala, como *MinMaxScaler, StandardScaler* y *RobustScaler*, así como transformaciones para encadenar la distribución de las variables de entrada como *QuantileTransformer* y *KBinsDiscretizer*. Finalmente, se utilizarán también transformaciones para remover dependencias lineales entre las variables de entrada como *PCA* y *TruncatedSVD*.

La clase *FeatureUnion* se puede usar para definir una lista de transformaciones para realizar y luego agregar (unir) sus resultados. Esto crea un nuevo *dataset* que posee un vasto número de columnas.

Luego, se puede crear un *Pipeline* con el *FeatureUnion* como el primer paso y el modelo de regresión logística como el último paso.

In [3]:
# data preparation as feature engineering for wine dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'
df = read_csv(url, header=None)
data = df.values
X, y = data[:, :-1], data[:, -1]
# minimally prepare dataset
X = X.astype('float')
y = LabelEncoder().fit_transform(y.astype('str'))
# transforms for the feature union
transforms = list()
transforms.append(('mms', MinMaxScaler()))
transforms.append(('ss', StandardScaler()))
transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))
# create the feature union
fu = FeatureUnion(transforms)
# define the model
model = LogisticRegression(solver='liblinear')
# define the pipeline
steps = list()
steps.append(('fu', fu))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)
# define the cross-validation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.968 (0.037)


En este caso, se alcanzó un rendimiento de 96.8 porciento, mejor que el *baseline*.

También, se puede utilizar *feature selection* para reducir las aproximadamente 80 *extracted features* a un subconjunto de aquellos más relevantes al modelo. Además de reducir la complejidad del modelo, también puede resultar en una mejora en el rendimiento al remover información irrelevante o redundante.

En este caso, se utilizará la **Recursive Feature Elimination (RFE)** para elegir los 15 *features* más relevantes. Luego, se puede agregar el RFE al *pipeline*, después del *FeatureUnion* y antes del modelo de regresión logística.

In [4]:
# data preparation as feature engineering with feature selection for wine dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'
df = read_csv(url, header=None)
data = df.values
X, y = data[:, :-1], data[:, -1]
# minimally prepare dataset
X = X.astype('float')
y = LabelEncoder().fit_transform(y.astype('str'))
# transforms for the feature union
transforms = list()
transforms.append(('mms', MinMaxScaler()))
transforms.append(('ss', StandardScaler()))
transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))
# create the feature union
fu = FeatureUnion(transforms)
# define the feature selection
rfe = RFE(estimator=LogisticRegression(solver='liblinear'), n_features_to_select=15)
# define the model
model = LogisticRegression(solver='liblinear')
# define the pipeline
steps = list()
steps.append(('fu', fu))
steps.append(('rfe', rfe))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)
# define the cross-validation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.989 (0.022)


## ¿Qué aprendí?

La forma de presentar los conceptos, en un *pipeline*, me parece que funciona muy bien con lo que se quiere obtener: una serie de pasos para navegar los datos sin tener que gastar demasiado tiempo o sin necesitar mucha experiencia. Creo que esto es particularmente útil cuando no se sabe cómo empezar a procesar los datos, pues puede ofrecer, como mínimo, impresiones iniciales sobre cuáles *features* conservar.

También, aprendí cómo funciona el RFE, que había sido mencionado en tutoriales anteriores. Me parece interesante cómo se pueden elegir hiperparámetros (por ejemplo, seleccionar los *K* features más importantes del *dataset*) para encontrar cuál punto genera los mejores resultados. Aunado a lo anterior, me gustó aprender el concepto de *baseline performance* que, aunque tenía esta idea intuitiva, no conocía la definición técnica para ella.

## ¿Qué me genera dudas?

Me interesa conocer cuáles técnicas, si las hay, se usan en conjunto o están relacionadas a *Recursive Feature Elimination*. Ha aparecido en varios tutoriales y parecer ser una técnica muy importante en la preparación de los datos.